# Development of raw data downloaders
##  ONS Estimates of the population for England and Wales

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.raw import download_data, data_filters

### Finding dataset ID and parameters using ONS API

In [59]:
import requests
import logging

logger = logging.getLogger(__name__)

In [57]:
ROOT_URL = "https://api.beta.ons.gov.uk/v1/"

In [63]:
def get_list_of_datasets():
    """Get list of all datasets available from API.
    Currently (August 2021), there are 41 available datasets.

    Returns
    -------
    list of dicts
        Metadata objects for each available dataset.
    """
    num_to_get = 100
    datasets = []

    offset = 0
    while len(datasets) < num_to_get:
        r = requests.get(ROOT_URL + "datasets", params={"offset": offset})
        results = r.json()
        [print(item.get("title")) for item in results.get("items")]
        datasets.extend(results.get("items"))
        num_retrieved = results.get("count")
        offset += num_retrieved
        if num_retrieved == 0:
            break
    print(f"\nFound {len(datasets)} datasets")
    return datasets

In [64]:
datasets = get_list_of_datasets()

Quarterly personal well-being estimates
Personal well-being estimates by local authority
Deaths registered weekly in England and Wales by region
Death registrations and occurrences by local authority and place of death
Death registrations and occurrences by health board and place of death
Deaths registered weekly in England and Wales by age and sex
UK spending on credit and debit cards
UK Business: Activity, Size and Location
Traffic Camera Activity
Trade in goods: country by commodity
Effects of Taxes and Benefits on Household Income
Suicide registrations in England and Wales by local authority
Sexual orientation by English regions and UK countries
Sexual orientation by age and sex
Retail sales index - large and small businesses
Retail sales index - all businesses
Retail sales index
Annual GDP for England, Wales and the English regions
Quarterly GDP for England, Wales and the English regions
Local authority ageing statistics, projected sex ratios for older people
Local authority agein

In [65]:
def get_dataset_by_name(datasets, target_name):
    """Get a dataset by matching on its title (or part of the title).
    Returns the first dataset object whose name contains the given target_name string.

    Parameters
    ----------
    datasets : List
        List of dataset objects
    target_name : str
        name (or partial name) of target dataset

    Returns
    -------
    dict
        Dataset object (or None, if no match is found)
    """
    for ds in datasets:
        if target_name.lower() in ds.get("title").lower():
            print(f"Found dataset '{ds.get('title')}'")
            return ds
    print(f"No dataset found containing '{target_name}'")
    return None

In [68]:
get_dataset_by_name(datasets, "Population")

Found dataset 'Population Estimates for UK, England and Wales, Scotland and Northern Ireland'


{'contacts': [{'email': 'pop.info@ons.gov.uk',
   'name': 'Neil Park',
   'telephone': '+44 (0)1329 444661'}],
 'description': 'Estimates of the usual resident population for the UK as at 30 June of the reference year. Provided by administrative area, single year of age and sex.',
 'keywords': ['Population'],
 'id': 'mid-year-pop-est',
 'last_updated': '2024-09-10T09:29:27.068Z',
 'license': 'Open Government Licence v3.0',
 'links': {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions'},
  'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3',
   'id': '3'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est'},
  'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/populationandmigration/populationestimates'}},
 'national_statistic': True,
 'next_release': 'Summer 2025',
 'publications': [{'href': 'https://www.ons.gov.uk/p

## Starting to build into a script

In [69]:
from src.data.raw import ons_api

In [70]:
datasets = ons_api.get_list_of_datasets()
datasets

[{'contacts': [{'email': 'qualityoflife@ons.gov.uk',
    'name': 'Will Shufflebottom, Owain Birrell, Pavan Bains and Geeta Kerai',
    'telephone': '+44 300 0671543'}],
  'description': 'Seasonally and non seasonally-adjusted quarterly estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety in the UK.',
  'keywords': ['well-being'],
  'id': 'wellbeing-quarterly',
  'last_updated': '2023-12-13T09:40:24.204Z',
  'links': {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions'},
   'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions/time-series/versions/9',
    'id': '9'},
   'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly'},
   'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/wellbeing'}},
  'methodologies': [{'href': 'https://www.ons.gov.uk/peoplepopulationandcommunity/wellbeing/methodologies/p

In [72]:
population_datasets = ons_api.get_dataset_by_name(datasets, "Population")
population_datasets

{'contacts': [{'email': 'pop.info@ons.gov.uk',
   'name': 'Neil Park',
   'telephone': '+44 (0)1329 444661'}],
 'description': 'Estimates of the usual resident population for the UK as at 30 June of the reference year. Provided by administrative area, single year of age and sex.',
 'keywords': ['Population'],
 'id': 'mid-year-pop-est',
 'last_updated': '2024-09-10T09:29:27.068Z',
 'license': 'Open Government Licence v3.0',
 'links': {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions'},
  'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3',
   'id': '3'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est'},
  'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/populationandmigration/populationestimates'}},
 'national_statistic': True,
 'next_release': 'Summer 2025',
 'publications': [{'href': 'https://www.ons.gov.uk/p

In [73]:
latest_edition = ons_api.get_edition(population_datasets)
latest_edition

'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3'

In [74]:
available_dimensions = ons_api.get_dimensions(latest_edition)
available_dimensions

{'age': {'total': 'Total',
  '0': '0',
  '1': '1',
  '2': '2',
  '3': '3',
  '4': '4',
  '5': '5',
  '6': '6',
  '7': '7',
  '8': '8',
  '9': '9',
  '10': '10',
  '11': '11',
  '12': '12',
  '13': '13',
  '14': '14',
  '15': '15',
  '16': '16',
  '17': '17',
  '18': '18',
  '19': '19',
  '20': '20',
  '21': '21',
  '22': '22',
  '23': '23',
  '24': '24',
  '25': '25',
  '26': '26',
  '27': '27',
  '28': '28',
  '29': '29',
  '30': '30',
  '31': '31',
  '32': '32',
  '33': '33',
  '34': '34',
  '35': '35',
  '36': '36',
  '37': '37',
  '38': '38',
  '39': '39',
  '40': '40',
  '41': '41',
  '42': '42',
  '43': '43',
  '44': '44',
  '45': '45',
  '46': '46',
  '47': '47',
  '48': '48'},
 'geography': {'K04000001': 'ENGLAND AND WALES',
  'E92000001': 'ENGLAND',
  'W92000004': 'WALES',
  'E12000001': 'NORTH EAST',
  'E12000002': 'NORTH WEST',
  'E12000003': 'YORKSHIRE AND THE HUMBER',
  'E12000004': 'EAST MIDLANDS',
  'E12000005': 'WEST MIDLANDS',
  'E12000006': 'EAST',
  'E12000007': 'LON

In [81]:
chosen_dimensions = (
    ons_api.choose_dimensions(
        available_dimensions,
        overrides={
            'age': '*',
            'geography': '*',
            'sex': '*',
            # 'sex': ['female', 'male'],
            'year': '2023'
        }
        )
        )
chosen_dimensions

{'age': '*', 'geography': '*', 'sex': '*', 'time': '*', 'year': '2023'}

In [85]:
population_url = ons_api.main()
population_url

'https://api.beta.ons.gov.uk/v1/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3'

In [86]:
from src.data.raw import download_data

In [90]:
download_data.get_population_data()

TypeError: string indices must be integers, not 'str'

In [89]:
download_data.get_outcomes_by_offence_data()

In [92]:
url = ons_api.main()
response = requests.get(url, timeout=10)
data = response.json()

In [93]:
data

{'alerts': [],
 'collection_id': 'cmdmidyearpopest-a468ce7ac2bcafa300afee011f6d1967566b66dcafc2d582f1305511a663ba06',
 'dimensions': [{'label': 'Time',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/calendar-years',
   'id': 'calendar-years',
   'name': 'time'},
  {'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/administrative-geography',
   'id': 'administrative-geography',
   'name': 'geography'},
  {'label': 'Sex',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/sex',
   'id': 'sex',
   'name': 'sex'},
  {'label': 'Age',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/single-year-of-age',
   'id': 'single-year-of-age',
   'name': 'age'}],
 'downloads': {'csv': {'href': 'https://download.ons.gov.uk

In [105]:
downloads = data['downloads']
downloads

{'csv': {'href': 'https://download.ons.gov.uk/downloads/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3.csv',
  'size': '68413226'},
 'csvw': {'href': 'https://download.ons.gov.uk/downloads/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3.csv-metadata.json',
  'size': '1639'}}

In [103]:
[download for download in downloads if download == "csv"]

['csv']

In [106]:
type(downloads)

dict

In [108]:
href = downloads.get('csv', {}).get('href')
href

'https://download.ons.gov.uk/downloads/datasets/mid-year-pop-est/editions/mid-2022-england-wales/versions/3.csv'

In [110]:
download_data.get_population_data()

In [111]:
url = ons_api.main()
download_data.fetch_json(url)

{'alerts': [],
 'collection_id': 'cmdmidyearpopest-a468ce7ac2bcafa300afee011f6d1967566b66dcafc2d582f1305511a663ba06',
 'dimensions': [{'label': 'Time',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/calendar-years',
   'id': 'calendar-years',
   'name': 'time'},
  {'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/administrative-geography',
   'id': 'administrative-geography',
   'name': 'geography'},
  {'label': 'Sex',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/sex',
   'id': 'sex',
   'name': 'sex'},
  {'label': 'Age',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/single-year-of-age',
   'id': 'single-year-of-age',
   'name': 'age'}],
 'downloads': {'csv': {'href': 'https://download.ons.gov.uk